In [ ]:
import sys
import time
import json
import heapq
import random
from collections import deque

# ---------------------------------------------
# State Representation and Utility Functions
# ---------------------------------------------

def is_valid(state):
    """
    Check if the given state is valid.
    The goat cannot be left alone with the wolf or the cabbage.
    """
    farmer, wolf, goat, cabbage, boat = state
    # Goat and wolf alone
    if goat == wolf and farmer != goat:
        return False
    # Goat and cabbage alone
    if goat == cabbage and farmer != goat:
        return False
    return True

def get_successors(state):
    """
    Generate all valid successor states from the current state.
    Returns a list of (next_state, action_description) tuples.
    """
    farmer, wolf, goat, cabbage, boat = state
    moves = []
    # Farmer crosses alone
    new_boat = 'R' if boat == 'L' else 'L'
    new_state = (new_boat if farmer == boat else farmer, 
                 wolf, goat, cabbage, new_boat)
    if is_valid(new_state):
        moves.append((new_state, "Farmer crosses alone"))
    # Farmer takes wolf
    if farmer == wolf == boat:
        new_state = (new_boat, new_boat, goat, cabbage, new_boat)
        if is_valid(new_state):
            moves.append((new_state, "Farmer takes Wolf"))
    # Farmer takes goat
    if farmer == goat == boat:
        new_state = (new_boat, wolf, new_boat, cabbage, new_boat)
        if is_valid(new_state):
            moves.append((new_state, "Farmer takes Goat"))
    # Farmer takes cabbage
    if farmer == cabbage == boat:
        new_state = (new_boat, wolf, goat, new_boat, new_boat)
        if is_valid(new_state):
            moves.append((new_state, "Farmer takes Cabbage"))
    return moves

def state_to_str(state):
    """
    Convert a state tuple to a readable string.
    """
    names = ['Farmer', 'Wolf', 'Goat', 'Cabbage', 'Boat']
    return ', '.join(f"{name}:{pos}" for name, pos in zip(names, state))

def read_input(filename):
    """
    Read parameters from a JSON input file.
    """
    with open(filename, 'r') as f:
        params = json.load(f)
    return params

def write_log(logfile, text):
    """
    Append a line of text to the log file.
    """
    with open(logfile, 'a') as f:
        f.write(text + '\n')

def print_path(path, actions, logfile):
    """
    Print the solution path and actions to the log file.
    """
    write_log(logfile, f"Solution found in {len(path)-1} steps:")
    for i, (state, action) in enumerate(zip(path[1:], actions)):
        write_log(logfile, f"Step {i+1}: {action} -> {state_to_str(state)}")

# ---------------------------------------------
# Breadth-First Search (BFS)
# ---------------------------------------------

def bfs(start, goal, logfile):
    """
    Breadth-First Search (BFS) implementation.
    Explores the shallowest nodes first using a queue (FIFO).
    """
    queue = deque()
    queue.append((start, [start], []))
    visited = set()
    step = 0
    while queue:
        state, path, actions = queue.popleft()
        write_log(logfile, f"BFS Step {step}: {state_to_str(state)}")
        if state == goal:
            return path, actions
        visited.add(state)
        for succ, action in get_successors(state):
            if succ not in visited:
                queue.append((succ, path + [succ], actions + [action]))
        step += 1
    return None, None

# ---------------------------------------------
# Depth-First Search (DFS)
# ---------------------------------------------

def dfs(start, goal, logfile, depth_limit=None):
    """
    Depth-First Search (DFS) implementation.
    Explores as far as possible along each branch before backtracking using a stack (LIFO).
    If depth_limit is set, performs Depth-Limited Search (DLS).
    """
    stack = [(start, [start], [], 0)]
    visited = set()
    step = 0
    while stack:
        state, path, actions, depth = stack.pop()
        write_log(logfile, f"DFS Step {step}: {state_to_str(state)} (depth {depth})")
        if state == goal:
            return path, actions
        if depth_limit is not None and depth >= depth_limit:
            continue
        visited.add(state)
        for succ, action in get_successors(state):
            if succ not in visited:
                stack.append((succ, path + [succ], actions + [action], depth + 1))
        step += 1
    return None, None

def dls(start, goal, logfile, limit):
    """
    Depth-Limited Search (DLS) implementation.
    Calls DFS with a depth limit.
    """
    return dfs(start, goal, logfile, depth_limit=limit)

# ---------------------------------------------
# Iterative Deepening Search (IDS)
# ---------------------------------------------

def ids(start, goal, logfile, max_depth):
    """
    Iterative Deepening Search (IDS) implementation.
    Repeatedly applies DLS with increasing depth limits.
    """
    for depth in range(max_depth + 1):
        write_log(logfile, f"IDS: Trying depth {depth}")
        path, actions = dfs(start, goal, logfile, depth_limit=depth)
        if path:
            return path, actions
    return None, None

# ---------------------------------------------
# Uniform Cost Search (UCS)
# ---------------------------------------------

def ucs(start, goal, logfile, cost_fn):
    """
    Uniform Cost Search (UCS) implementation.
    Expands the least-cost node using a priority queue.
    """
    heap = []
    heapq.heappush(heap, (0, start, [start], []))
    visited = set()
    step = 0
    while heap:
        cost, state, path, actions = heapq.heappop(heap)
        write_log(logfile, f"UCS Step {step}: {state_to_str(state)} (cost {cost})")
        if state == goal:
            return path, actions
        visited.add(state)
        for succ, action in get_successors(state):
            if succ not in visited:
                heapq.heappush(heap, (cost + cost_fn(state, succ), succ, path + [succ], actions + [action]))
        step += 1
    return None, None

# ---------------------------------------------
# Iterative/Randomized Local Search (ILS)
# ---------------------------------------------

def ils(start, goal, logfile, max_restarts):
    """
    Iterative/Randomized Local Search (ILS) implementation.
    Performs multiple DFS runs with randomization and restarts.
    """
    best_path = None
    best_actions = None
    for restart in range(max_restarts):
        write_log(logfile, f"ILS: Restart {restart}")
        stack = [(start, [start], [], 0)]
        visited = set()
        while stack:
            state, path, actions, depth = stack.pop()
            if state == goal:
                if not best_path or len(path) < len(best_path):
                    best_path = path
                    best_actions = actions
                break
            visited.add(state)
            successors = get_successors(state)
            random.shuffle(successors)
            for succ, action in successors:
                if succ not in visited:
                    stack.append((succ, path + [succ], actions + [action], depth + 1))
    return best_path, best_actions

# ---------------------------------------------
# Main Function
# ---------------------------------------------

def main():
    if len(sys.argv) != 3:
        print("Usage: python river_crossing_full_search.py <algorithm> <input_file>")
        sys.exit(1)
    algorithm = sys.argv[1].lower()
    input_file = sys.argv[2]
    params = read_input(input_file)
    start = tuple(params["start"])
    goal = tuple(params["goal"])
    logfile = params["logfile"]
    # Clear log file at the start
    open(logfile, 'w').close()
    t0 = time.time()
    if algorithm == "bfs":
        path, actions = bfs(start, goal, logfile)
    elif algorithm == "dfs":
        path, actions = dfs(start, goal, logfile)
    elif algorithm == "dls":
        path, actions = dls(start, goal, logfile, params["depth_limit"])
    elif algorithm == "ids":
        path, actions = ids(start, goal, logfile, params["max_depth"])
    elif algorithm == "ucs":
        def cost_fn(s1, s2): return params.get("step_cost", 1)
        path, actions = ucs(start, goal, logfile, cost_fn)
    elif algorithm == "ils":
        path, actions = ils(start, goal, logfile, params["max_restarts"])
    else:
        print("Unknown algorithm")
        sys.exit(1)
    t1 = time.time()
    if path:
        print_path(path, actions, logfile)
    else:
        write_log(logfile, "No solution found.")
    write_log(logfile, f"Time taken: {t1-t0:.4f} seconds")

if __name__ == "__main__":
    main()